In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import sklearn
import pickle
import matplotlib.pyplot as plt

from tqdm import tqdm
from DataLoader import DataLoader
from Inference import Inference
from Predictor import Predictor

In [3]:
adata_path = '../../Data/sc_training.h5ad'

# Infering
n_infer_instances = 1000 # Number of instances used for infering the network. -1 = all 
n_infer_estimators = 4 # Number of trees in random forest for infering the network
max_depth_infer = 100 # max depth of random forest for infering the network
importance_threshold = 1

# Predicting
n_train_instances = 1000 # Number of instances used training predictor models. -1 = all 
n_train_estimators = 4 # Number of trees in random forest training predictor models
max_depth_train = 100 # max depth of random forest training predictor models

# Classification
n_features = 1000
n_components = 100
n_neighbors = 10

# Dataloader

In [4]:
dataloader = DataLoader()
dataloader.load_data(adata_path)


Loading data from h5ad file.


e:\Documenten\GitHub\Cancer-Immunotherapy-Data-Science-Grand-Challenge\Challenge1\code\DataLoader.py:15: FutureWarning: The behavior of .astype from SparseDtype to a non-sparse dtype is deprecated. In a future version, this will return a non-sparse array with the requested dtype. To retain the old behavior, use `obj.astype(SparseDtype(dtype))`
  self.gene_expressions = pd.DataFrame.sparse.from_spmatrix(self.adata.X, columns=self.adata.var_names).astype(float).to_numpy()



Succesfully loaded the data.


In [45]:
# Print state proportions of random knockout
conditions = dataloader.adata.obs['condition'].to_numpy()
gene = np.random.choice(conditions)
proportions = dataloader.get_state_proportions_of_condition(gene)
print(gene, proportions)

Sp100 [0.02447552 0.28846154 0.28146853 0.39160839 0.01398601]


# Inference

In [5]:
inference = Inference(dataloader)
inference.load_network('GRNetwork')


Loading genetic regulatory network.


In [6]:
inference.set_importance_threshold(0.011)
genes = ['Aqr', 'Bach2', 'Bhlhe40', 'Ets1', 'Fosb', 'Mafk', 'Stat3']
for gene in genes:
    order = inference.get_knockout_order(gene)
    print(gene, len(order))

Aqr 19
Bach2 3
Bhlhe40 3660
Ets1 6559
Fosb 226
Mafk 5
Stat3 1486


# Predictor

In [6]:
predictor = Predictor(dataloader, inference)
predictor.fit(n_features, n_components, n_neighbors)


Fitting predictor.


In [7]:
proportions = dataloader.get_state_proportions_of_condition('Unperturbed')
print(f'Unperturbed: {proportions}')

genes = ['Aqr', 'Bach2', 'Bhlhe40', 'Ets1', 'Fosb', 'Mafk', 'Stat3']
for gene in genes:
    predictions = predictor.predict_knockout_effect(gene, n_components, n_neighbors)
    print(f'{gene}, {predictions}')

Unperturbed: [0.06749699 0.20972278 0.31337887 0.39212535 0.01727601]

Predicting state proportions after knockout of Aqr.
Aqr, [0.06830052 0.1982724  0.31619124 0.40437927 0.01285657]

Predicting state proportions after knockout of Bach2.
Bach2, [0.06830052 0.1982724  0.31619124 0.40437927 0.01285657]

Predicting state proportions after knockout of Bhlhe40.
Bhlhe40, [0.06870229 0.19967859 0.31337887 0.40538369 0.01285657]

Predicting state proportions after knockout of Ets1.
Ets1, [0.06850141 0.19726798 0.31679389 0.40458015 0.01285657]

Predicting state proportions after knockout of Fosb.
Fosb, [0.06830052 0.1982724  0.31619124 0.40437927 0.01285657]

Predicting state proportions after knockout of Mafk.
Mafk, [0.06830052 0.1982724  0.31619124 0.40437927 0.01285657]

Predicting state proportions after knockout of Stat3.
Stat3, [0.06850141 0.19787063 0.31639213 0.40478104 0.0124548 ]


In [41]:
gene = 'Dvl2'
proportions = dataloader.get_state_proportions_of_condition(gene)
predictions = predictor.predict_knockout_effect(gene, n_components, n_neighbors)
mae = np.mean(abs(proportions - predictions))
print(gene)
print(proportions)
print(predictions)
print(mae)


Predicting state proportions after knockout of Dvl2.
Dvl2
[0.02439024 0.11550851 0.30418776 0.54947078 0.00644271]
[0.06830052 0.20208919 0.31538771 0.40076336 0.01345922]
0.05948296757920769
